In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,plot_confusion_matrix
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet

In [4]:
df = pd.read_csv("r_dataisbeautiful_posts.csv")

In [5]:
df.isna().sum()

id                            0
title                         2
score                         0
author                        0
author_flair_text        314962
removed_by               342498
total_awards_received    280012
awarders                 301362
created_utc                   0
full_link                     0
num_comments                  0
over_18                       0
dtype: int64

In [6]:
del df['author_flair_text']
del df['removed_by']
del df['total_awards_received']
del df['awarders']
del df['id']
del df['created_utc']
del df['full_link']
del df['over_18']

In [7]:
df.head()

,title,score,author,num_comments
0,Looking at how easy/hard the goals scored by s...,1,sthsthanothersth,1
1,Looking at how easy/hard the goals scored by s...,1,sthsthanothersth,1
2,[OC] Yet another Covid-19 visualization,1,dslfdslj,2
3,My phone shows some serious steps increase aft...,1,Paulinkica,1
4,My phone shows some serious steps increase aft...,1,Paulinkica,2


In [8]:
del df['author']

In [15]:
363012/2

181506.0

In [18]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [19]:
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [20]:
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    final_text = []
    try:
        for i in text.split():
            if i.strip().lower() not in stop:
                pos = pos_tag([i.strip()])
                word = lemmatizer.lemmatize(i.strip(),get_simple_pos(pos[0][1]))
                final_text.append(word.lower())
        return final_text
    except:
        print('hehe')  

In [34]:
df.title = df.title.apply(lemmatize_words)

AttributeError: 'float' object has no attribute 'split'

In [ ]:
def join_text(text):
    string = ''
    for i in text:
        string += i.strip() +' '
    return string  

In [ ]:
df.title = df.title.apply(join_text)

In [35]:
train_message = df.title[:181506]
test_message = df.title[181506:]
train_category = df.score[:181506]
test_category = df.score[181506:]

In [36]:
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,2))
#transformed train reviews
cv_train_reviews=cv.fit_transform(train_message)
#transformed test reviews
cv_test_reviews=cv.transform(test_message)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,2))
#transformed train reviews
tv_train_reviews=tv.fit_transform(train_message)
#transformed test reviews
tv_test_reviews=tv.transform(test_message)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

In [ ]:
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for Bag of words
lr_bow=lr.fit(cv_train_reviews,train_category)
print(lr_bow)
#Fitting the model for tfidf features
lr_tfidf=lr.fit(tv_train_reviews,train_category)
print(lr_tfidf)

In [ ]:
lr_bow_predict=lr.predict(cv_test_reviews)
##Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tv_test_reviews)

In [ ]:
#Accuracy score for bag of words
lr_bow_score=accuracy_score(test_category,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)
#Accuracy score for tfidf features
lr_tfidf_score=accuracy_score(test_category,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)